# 교차 검증, 하이퍼파라미터 튜닝
- 가장 좋은 모델 선정

In [1]:
# 필요한 라이브러리 임포트
import pandas as pd  # 데이터 처리를 위한 pandas
import numpy as np   # 수치 계산을 위한 numpy
from sklearn.ensemble import RandomForestClassifier  # 랜덤 포레스트 분류기
from sklearn.svm import SVC  # 서포트 벡터 머신
from sklearn.linear_model import LogisticRegression  # 로지스틱 회귀
from sklearn.model_selection import train_test_split, cross_val_score  # 데이터 분할 및 교차 검증
from sklearn.metrics import accuracy_score  # 정확도 평가 지표

# 데이터 가져오기
data = pd.read_csv("train.csv")
data.head()

# 모델링에 사용할 특성(feature)과 타겟(target) 변수 선택
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']  # 승객 등급, 성별, 나이, 동반자 수, 요금, 탑승 항구
target = 'Survived'  # 생존 여부 (0: 사망, 1: 생존)

data['Survived'].value_counts() # 비율이 5:5 (x) ==> 클래스 불균형, Class Imbalanced

Survived
0    549
1    342
Name: count, dtype: int64

In [2]:
# 결측치 확인
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


# 데이터 전처리
- 고려사항
  + 전체 데이터에 해도 무방한가? 아니면 훈련데이터에만 적용해야 하는가?
  + 범주 데이터는 두가지 선택지 : One-Hot Encoder, Label Encoder (0, 1, 2) / 탐색적 데이터 분석 & 사회적인 통념 기반으로 결정

In [3]:
# data['Sex'].unique()
data['Sex'] = data['Sex'].map({'male' : 0, 'female' : 1}) # 성별, 남성 = 0, 여성 = 1
data['Embarked'] = data['Embarked'].map({'C' : 0, 'Q' : 1, 'S' : 2})

data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,2.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,0.0
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,2.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,2.0
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,2.0


In [5]:
data['Embarked'].value_counts()

Embarked
2.0    644
0.0    168
1.0     77
Name: count, dtype: int64

# 데이터셋 분리

In [6]:
# 현재 고려하지 않은 feature 3개 : ID, Name, Cabin
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']  # 승객 등급, 성별, 나이, 동반자 수, 요금, 탑승 항구
target = 'Survived'  # 생존 여부 (0: 사망, 1: 생존)

X = data[features] # 특성 데이터
y = data[target] # 타겟 데이터

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state = 42, stratify = y
)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((668, 7), (223, 7), (668,), (223,))

# 피처 엔지니어링

In [23]:
# 훈련데이터의 정보를 이용해서 테스트 데이터에 적용
# 훈련데이터에서 결측치를 채울 대표값 계산
age_mean = round(X_train['Age'].mean(), 0)                # 승객 등급의 최빈값
embarked_mode = X_train['Embarked'].mode()[0]             # 탑승 항구의 최빈값

age_mean, embarked_mode

# 훈련셋에서 결측치를 채울 대표값 계산
age_mean = X_train['Age'].mean()  # 나이의 평균값
fare_mean = X_train['Fare'].mean()  # 요금의 평균값
pclass_mode = X_train['Pclass'].mode()[0]  # 승객 등급의 최빈값
embarked_mode = X_train['Embarked'].mode()[0]  # 탑승 항구의 최빈값

In [24]:
# 훈련데이터 적용
X_train['Age'] = X_train['Age'].fillna(age_mean)
X_train['Embarked'] = X_train['Embarked'].fillna(embarked_mode)

# 테스트 데이터 적용
X_test['Age'] = X_test['Age'].fillna(age_mean)
X_test['Embarked'] = X_test['Embarked'].fillna(embarked_mode)

- 정규화, 표준화, One-Hot Encoding 고려해야 하나, 여기서는 안함
  + 이유 : 연속형 데이터로 볼만한 것이 없음 / 이산형 데이터(Count 데이터)가 대부분

# 모델링
- 각 모델별 하이퍼파라미터 후보군 정의
- 교차 검증
- 가장 최고의 모델을 선정

In [25]:
# 사용할 모델 정의
models = {
    'RandomForest': RandomForestClassifier(random_state=42),  # 랜덤 포레스트 분류기
    'SVM': SVC(random_state=42, probability=True),  # 서포트 벡터 머신
    'LogisticRegression': LogisticRegression(random_state=42)  # 로지스틱 회귀
}

# 각 모델별 하이퍼파라미터 후보군 정의
param_grid = {
    'RandomForest': [
        {'n_estimators': 100, 'max_depth': None},  # 트리 100개, 깊이 제한 없음
        {'n_estimators': 200, 'max_depth': 5}      # 트리 200개, 최대 깊이 5
    ],
    'SVM': [
        {'C': 1.0, 'kernel': 'rbf'},    # RBF 커널, C=1.0
        {'C': 0.5, 'kernel': 'linear'}  # 선형 커널, C=0.5
    ],
    'LogisticRegression': [
        {'C': 1.0, 'max_iter': 1000},  # 기본 설정
        {'C': 0.1, 'max_iter': 1000}   # 더 강한 정규화
    ]
}

# 교차 검증을 통한 최적 모델 선택
best_score = 0 # 최고 성능 점수
best_model_name = None # 최고 성능 모델 이름
best_model = None # 최고 성능 모델 객체

# 각 모델과 하이퍼파라미터 조합에 대해 교차검증 수행
for model_name, model in models.items():
    print(f"\n--- Testing {model_name} ---")
    for params in param_grid[model_name]:
        model.set_params(**params)  # 하이퍼파라미터 설정
        cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')  # 5-fold 교차검증
        mean_cv = np.mean(cv_scores)  # 평균 교차검증 점수
        print(f"Params: {params}, CV Accuracy: {mean_cv:.4f}")
        
        # 최고 성능 모델 업데이트
        if mean_cv > best_score:
            best_score = mean_cv
            best_model_name = model_name
            best_model = model.set_params(**params)

# 최종 선택된 모델로 테스트셋 평가
best_model.fit(X_train, y_train)  # 최적 모델 학습
y_pred = best_model.predict(X_test)  # 테스트셋 예측
test_acc = accuracy_score(y_test, y_pred)  # 테스트셋 정확도 계산

# 최종 결과 출력
print(f"\nBest Model: {best_model_name}")
print(f"Best CV Score: {best_score:.4f}")
print(f"Test Set Accuracy: {test_acc:.4f}")


--- Testing RandomForest ---
Params: {'n_estimators': 100, 'max_depth': None}, CV Accuracy: 0.8249
Params: {'n_estimators': 200, 'max_depth': 5}, CV Accuracy: 0.8264

--- Testing SVM ---
Params: {'C': 1.0, 'kernel': 'rbf'}, CV Accuracy: 0.6931
Params: {'C': 0.5, 'kernel': 'linear'}, CV Accuracy: 0.7934

--- Testing LogisticRegression ---
Params: {'C': 1.0, 'max_iter': 1000}, CV Accuracy: 0.8024
Params: {'C': 0.1, 'max_iter': 1000}, CV Accuracy: 0.8084

Best Model: RandomForest
Best CV Score: 0.8264
Test Set Accuracy: 0.7758


# 각 알고리즘 개별적으로 공부하는 건 불가능
- 알고리즘에 집중하지 말고, 설계에 집중을 해라
- 설계 : 교차검증 & 하이퍼파라미터 튜닝 + 데이터 가공
- 시나리오를 구성

# Decision Tree는 부등호로 구분
- Gini, Entropy를 독립변수 구분

- RandomForest ==> Decision Tree의 확장판
 + tree1 : 사망
 + tree2 : 생존
 + tree3 : 사망
   - 결론 : 사망2, 생존1, 이 모델은 사망으로 판단

In [28]:
!pip install joblib

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Program Files\Python313\python.exe -m pip install --upgrade pip


In [29]:
import joblib
import json

# 전처리 정보 저장 & 모델 저장 내보내기
preprocessing_info = {
    'age_mean': float(age_mean),
    'fare_mean': float(fare_mean),
    'pclass_mode': int(pclass_mode),
    'embarked_mode': int(embarked_mode),
    'features': features,
    'sex_mapping': {'male': 0, 'female': 1},
    'embarked_mapping': {'C': 0, 'Q': 1, 'S': 2}
}

# 모델과 전처리 정보 저장
joblib.dump(best_model, 'titanic_model.joblib')

# 훈련데이터의 정보를 추가했다.
with open('preprocessing_info.json', 'w') as f:
    json.dump(preprocessing_info, f)

# 최종 결과 출력
print(f"\nBest Model: {best_model_name}")
print(f"Best CV Score: {best_score:.4f}")
print(f"Test Set Accuracy: {test_acc:.4f}")
print("\nModel and preprocessing information have been saved.")


Best Model: RandomForest
Best CV Score: 0.8264
Test Set Accuracy: 0.7758

Model and preprocessing information have been saved.
